In [4]:
import sys
sys.path.insert(0, "/home/alexp/NBA_Models/src/scripts/")

In [5]:
from data.transform_data import transform
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [6]:
path = "/home/alexp/NBA_Models/sqlite/db/nba_data.db"
conn = sqlite3.connect(path)

t = transform(conn,2013,2023)
clean = t.transform(type='agg_boxscores')
scaler = t.standard_scalar(clean)
lable_enc = t.label_encoder(scaler,['SEASON','TEAM_ABBREVIATION'])
merged_data = lable_enc

In [7]:
WL_corr = pd.DataFrame(clean.corr()['WL']).reset_index()

In [8]:
import altair as alt
# Define two single-value sliders for the lower and upper bounds
min_slider = alt.binding_range(min=-1, max=1, step=0.01, name='Min WL')
max_slider = alt.binding_range(min=-1, max=1, step=0.01, name='Max WL')

# Define the selection for the lower and upper bounds
select_wl_min = alt.selection_single(bind=min_slider, name='Min WL', fields=['WL_min'])
select_wl_max = alt.selection_single(bind=max_slider, name='Max WL', fields=['WL_max'])

# Create the bar chart with the selection filter applied
bar_chart = alt.Chart(WL_corr).mark_bar().encode(
    x='index:N',
    y='WL:Q',
    color=alt.Color('WL', scale=alt.Scale(range=['red', 'green'])
    )
).add_selection(
    select_wl_min,
    select_wl_max
).transform_filter(
    'datum.WL <= Min_WL_WL_min && datum.WL >= Max_WL_WL_max'
)

# Display the chart
bar_chart.display()

/home/alexp/miniconda3/envs/tf/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/home/alexp/miniconda3/envs/tf/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

In [9]:
WL_pos_corr = WL_corr[(WL_corr['WL'] > .1)]
WL_neg_corr = WL_corr[(WL_corr['WL'] < -.1)]
WL_high_corr = pd.concat([WL_pos_corr,WL_neg_corr])
filter_features = ['TEAM_ABBREVIATION','SEASON','HOME_GAME']+list(WL_high_corr['index'])
data = merged_data[filter_features]
data

,TEAM_ABBREVIATION,SEASON,HOME_GAME,HOME_GAME,WL,FG2M,FG3M_game,FTM_game,FTA_game,DREB_game,...,AST_players,STL_players,BLK_players,PTS_players,PLUS_MINUS_players,NBA_FANTASY_PTS,DD2,E_DEF_RATING,DEF_RATING,BLKA
0,12,0,0,0,0,0.717760,-0.687806,-0.755589,0.028747,-0.663832,...,2.700000,1.100000,0.400000,10.300000,-6.500000,22.050000,0.200000,109.1,117.2,0.600000
1,13,0,1,1,1,-0.227655,0.774284,0.079870,0.714272,0.070579,...,2.090909,0.727273,0.545455,10.545455,5.909091,21.445455,0.090909,103.9,105.1,0.363636
2,21,0,0,0,0,-0.416738,-0.444124,-1.925230,-1.753615,-1.398243,...,1.545455,0.909091,0.545455,7.909091,-4.545455,17.300000,0.000000,101.0,103.2,1.636364
3,11,0,1,1,1,-0.416738,-0.931487,0.748237,1.262691,0.070579,...,1.545455,0.363636,1.636364,8.818182,4.545455,20.118182,0.000000,84.1,92.6,0.545455
4,29,0,0,0,0,-0.983987,-0.200442,1.082420,1.262691,-1.581846,...,2.090909,0.454545,0.272727,9.272727,-5.000000,17.281818,0.181818,113.5,115.3,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22177,3,10,0,0,0,-0.605821,-0.200442,-1.423955,-1.479406,-0.663832,...,1.818182,0.636364,0.090909,8.272727,-5.909091,16.763636,0.000000,112.5,111.8,0.545455
22178,18,10,1,1,0,-1.551236,0.286921,-0.087222,0.028747,-0.847435,...,1.833333,0.750000,0.416667,7.916667,-6.666667,17.716667,0.083333,115.9,118.1,0.583333
22179,12,10,0,0,1,-0.794904,1.017966,-0.254314,-0.382567,1.172195,...,2.153846,0.461538,0.538462,8.538462,6.153846,18.307692,0.000000,97.8,101.1,0.384615
22180,17,10,0,0,1,0.528677,0.774284,-0.254314,-0.382567,1.723003,...,2.214286,0.857143,0.500000,8.714286,9.642857,19.564286,0.071429,90.8,95.0,0.142857
